# I) Installations et imports

In [1]:
# login : eleveenmi@gmail.com  (i=1,2,3,..)
# mwd : pwdeleveEnmi (i=1,2,3,..)

# Google sheet
!pip install --upgrade gspread gspread_dataframe

from google.colab import auth
from google.auth import default
import gspread
from google.colab import drive


import re

def text2labels(text):
    # Normalize spaces and underscores
    cleaned = re.sub(r'[\s_]+', ' ', text.strip())

    # Match tokens like C45, h1200, etc.
    matches = re.findall(r'\b([cnh])(\d+)\b', cleaned, flags=re.IGNORECASE)

    if not matches:
        return "SVP repecter le format: Cx Ny Hz avec X,Y,Z des entiers positifs"

    label_dict = {}
    key_map = {'c': 'Classe', 'n': 'Nebul', 'h': 'Hauteur'}

    for key, val in matches:
        val = int(val)
        if val <= 0:
            return "SVP repecter le format: Cx Ny Hz avec X,Y,Z des entiers positifs"
        if key.lower() == 'n' and val > 8:
            return "La nébulosité (N) doit être un entier entre 1 et 8"
        label_dict[key_map[key.lower()]] = val

    return label_dict


def send_labels(dict_labels, filename_to_labels):
    for filename, labels in filename_to_labels.items():
        if isinstance(labels, str):
            return labels  # Return the error immediately
        if filename not in dict_labels:
            return f"l'image n'est pas dans le dictionnaire"
        dict_labels[filename].update(labels)
    return dict_labels


def build_initial_dict_from_images(root_folder='images_to_label'):
    image_data = {}
    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.jpg'):
                image_data[filename] = {'Classe': None, 'Nebul': None, 'Hauteur': None}
    return image_data



# II) Authentification

In [2]:
# pour colab :
auth.authenticate_user()
# pour accéder aux fichiers google (google sheet)
creds, _ = default()  # or use service account
gc = gspread.authorize(creds)

#
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

# III) Récupération des images :

# IV) Récupération d'une partie des labels :

In [ ]:
# Open by ID or URL
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KT4RXpX3FkR2APlGXhD54euEpPWHbFZKrxoWdFzeqog")
worksheet = sh.sheet1
data = worksheet.get_all_records()


In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

,Image,Classe,Nebul,Hauteur
0,202209071633_S.jpg,41,4,2
1,202107301651_S.jpg,,,
2,202108161555_S.jpg,,,
3,202012042334_S.jpg,,,


# V) Annotation

In [ ]:
# Si aucun label posé :
dict_labels = build_initial_dict_from_images()
print(dict_labels)